# TP réseaux 2021/2022 - TP 8 : Découverte d'IPv6

Cette séance est une introduction à IPv6. On s'intéresse à l'adressage ([RFC 4291](#https://datatracker.ietf.org/doc/html/rfc4291)), à l'autoconfiguration et au routage statique. Il s'agit d'une découverte par l'observation et quelques configurations qui sont guidées ci-dessous.

## 1. Les adresses IPv6

Dans cette première partie, il s'agit d'observer les adresses IPv6 et de réaliser un réseau constitué de 2 sous réseaux contenant chacun 2 machines et d'un routeur avec une interface dans chaque réseau. Vous allez configurer progressivement ce réseau sous la forme d'un lab Kathara. 

### 1.1 Les adresses locales

Créez un *lab.conf* afin de lancer deux machines alice et bob avec une interface réseau dans un domaine de collision **A**. Il est nécessaire d'activer IPv6 sur les machines et également de mettre à faux l'IP forwarding sur ces machines qui ne sont pas des routeurs. Par exemple pour alice vous aurez les lignes suivantes dans votre lab.

In [ ]:
alice[0]="A"
alice[ipv6]="True"
alice[sysctl]="net.ipv6.conf.all.forwarding=0"
alice[image]="nopid/3ia:latest"


Après avoir lancé votre lab
1. observez la configuration de l'interface eth0 des machines alice et bob (commande ip addr)
2. observez le lien entre l'adresse MAC et l'adresse de lien local (scope link)

Arrêter le lab avec *Kathara wipe*.

## 1.2 Les adresses globales

Dans cette partie, on va rajouter un deuxième sous réseau et des adresses routables pour faire communiquer ces deux réseaux locaux. 

Complétez votre lab.conf afin de rajouter un domaine de collision **B** et deux machines charlie et dave avec eth0 dans ce domaine de collision.

Rajoutez un routeur **rtr** avec deux interfaces, eth0 dans le domaine de collision **A** et eth1 dans **B**. Attention il faut laisser la valeur de l'IP forwarding à 1 car il s'agit bien d'un routeur.

Pour la suite de la configuration, il s'agit de travailler directement dans les machines virtuelles une fois votre lab lancé. Et on choisit des adresses avec le préfixe réseau local 2001:ad38::/64 pour alice, bob et rtr (eth0), et  2001:ad54::/64 pour charlie, daisy et rtr (eth1). 

Finalisez la configuration des adresses et des tables de routage des différentes machines avec la commande *ip -6*


In [ ]:
ip -6 address add "@Ipv6"/64 dev eth0
ip -6 route add "@dest" via "@IPv6 de la passerelle"
ip -6 route show

Vérifiez que votre réseau est fonctionnel à l'aide de pings entre les machines avec les adresses *global scope*.

Le protocole ARP permet de faire le lien entre les adresses IPv4 (couche réseau) et les adresses MAC (couche liaison). En lançant *kathara wire A* ou *Kathara wire B* 


In [ ]:
Q5. Déterminez quels messages remplacent ARP pour IPv6 ? ARP => NDP (Neighbor Discorver Protocol)
Envoi une requete Neighbor Solicitation , reçoit en retour Neighbor Advertissement
Q6. Que fait la commande ip neigh ? Montre la table de routage (IP ET @MAC) des différents voisins

## 2. Autoconfiguration

Dans cette deuxième partie, vous allez utiliser deux lab labmini6.tgz et labdhcp6.tgz à charger sur la page sur Celene. Ils reprennent le réseau que vous avez construit dans la première partie mais ils contiennent également des éléments supplémentaires pour configurer deux protocoles associés à IPv6 pour l'autoconfiguration des machines. 

### 2.1 Autoconfiguration RADVD ([RFC 4862](https://datatracker.ietf.org/doc/html/rfc4862))

La première option d'autoconfiguration utilise un Router ADVertisement Daemon. Dans *labmini6*, **radvd** est configuré sur le routeur *rtr*.

Démarrez labmini6 par la commande 

In [ ]:
kathara lstart rtr bob daisy

et exécutez kathara wire A. Enfin, lancez la machine alice avec la commande suivante

In [ ]:
kathara lstart alice

Observez les échanges dans wireshark et lisez le contenu du fichier */etc/radvd.conf* 

Faîtes la même chose pour charlie sur le réseau **B**.

### 2.2 Autoconfiguration DHCP version 6 ([RFC 8415](https://datatracker.ietf.org/doc/html/rfc8415))

On va maintenant utiliser le couple **RADVD** et **DHCP** pour la configuration des machines.  

Lancez le lab de l'archive labdhcp6.tgz. Pour comprendre le protocole DHCP version 6 et son association avec RADVD suivez les étapes suivantes:

1. Vérifiez qu'alice a une adresse dans le réseau 2001:ad38::/64
2. Sur la machine *rtr* regardez le fichier /etc/radvd.conf et /etc/dhcp/dhcpd.conf
3. Lancez un wireshark sur le réseau **A** et exécutez la commande dhclient -6 eth0 sur bob. Observez les échanges et vérifiez que bob a désormais une adresse dans le réseau 2001:ad38::/64


A partir de vos observations sur alice et bob, modifiez *dhcpd6.conf* du routeur *rtr* pour qu'il puisse délivrer des adresses dans le réseau 2001:ad54::/64.

Après avoir relancé le serveur DHCP sur *rtr* avec la commande  

In [ ]:
/etc/init.d/isc-dhcp-server start

exécutez sur daisy et charlie la commande 

In [ ]:
dhclient -6 eth0 